In [2]:
import numpy as np 
import simpy 
import random

# [Problem 1: Deriving Average Wait Times for M/M/1 vs. M/M/C Queues]

# Average Waiting Time for M/M/1 Queues 

$E(W)_{M/M/1} = \frac{E(L)_{M/M/1}}{\lambda}$ (Little's Law)

$E(L)_{M/M/1} = \Sigma{n*p_n}$

$\text{Since } p_n = (1-\rho)*\rho^n \text{, then } E(L)_{M/M/1} = \Sigma {n*(1-\rho)*\rho^n}$

$\text{Recall } \Sigma \rho^n = \frac{1}{1-\rho} \text{by the Geometric Series}$

$\text{Then, we can differentiate both sides: } \frac{d}{d\rho} \Sigma \rho^n = \frac{d}{d\rho} \frac{1}{1-\rho} $

$\text{We get: } \Sigma n*\rho^{n-1} = \frac{1}{(1-\rho)^2}$ 

$\text{Therefore, } \Sigma n*p^n = \frac{\rho}{(1-\rho)^2}$

$\text{And } (1-\rho) * \Sigma n*\rho^n = \frac{\rho}{1-\rho}$

$\text{E(L)}_{M/M/1} = \frac{\rho}{1-\rho} = \frac{\frac{\lambda}{\mu}}{1-\rho}$

${E(W)}_{M/M/1} = \frac{{E(L)}_{M/M/1}}{\lambda} = \frac{1}{\mu*(1-\rho)}$

$\text{------------------------------------------------------------------}$

# Average Waiting Time for M/M/C Queues 
$E(W)_{M/M/C} = \frac{E(L)_{M/M/C}}{\lambda}$ (Little's Law)

$E(L)_{M/M/C} = \Sigma{n*p_{n+c}} = \Sigma{n*p^n*p_c}$

$ = p_c\Sigma{n*p^n}$

$ \text{Following logic from previous section, } \Sigma{n*p^n*p_c} = p_c * \frac{\rho}{(1-\rho)^2}$

$ = \frac{p_c}{1-\rho} * \frac{\rho}{1-\rho}$

$ \text{Since } \frac{p_c}{1-\rho} = \Pi_w \text{ and } \rho = \frac{\lambda}{n*\mu} \text{, then } E(L)_{M/M/C} = \Pi_w * \frac{\lambda}{(1-\rho)*n*\mu}$

$ \text{Therefore, } E(W)_{M/M/C} = \frac{\Pi_w}{(1-\rho)*n*\mu}$

$ \Pi_w \text{ is always <= 1 since it is the probability that a job has to wait. Therefore, } E(W)_{M/M/C} \text{ will always be lower than } E(W)_{M/M/1} \text{ by at least a factor of n.}$








# [Problem 2: DES Program to Verify Results]

In [ ]:
class MMCQueueSimulation: 
    def __init__(self,env,lambd,mu,n):
        '''
        Initializes object values in environment.

        Parameters: 
        ----------
        lambd: Arrival rate into the system
        mu: Capacity of each server
        n: Number of servers
        '''

        self.env = env
        self.lambd = lambd
        self.mu = mu
        self.n = n
        self.servers = simpy.Resource(env, capacity = n)
        self.waiting_times = []

    def one_service(self, name): 
        


    def average_wait_time(self, env, lambd, mu, n):
        '''
        Calculates average wait time of a job. 

        Parameters: 
        ----------
        lambd: Arrival rate into the system
        mu: Capacity of each server
        n: Number of servers

        Returns: 
        -------
        Average waiting time 
        '''


env = simpy.Environment()



    


